# Solutions for Exercise 2 - Unlinkable proofs using BBS+

This only holds the answer to the questions, and the solution to the last coding exercise.

---
## 1 - Questions for Setting up a corresponding JSON Schema

1. What is the format for `timeOfBirth`? And what is the limitations using that format? What else could the format be?

> The javascript class `Date` returns the milliseconds since the Unix Epoch
> (midnight of 1970-1-1) with the method `getTime()`.

> This means that times before the Unix Epoch will be negative, which can pose problems
> later on with Zero Knowledge Proofs.
> Depending on the precision of the number, this might also overflow, as the number of
> milliseconds since the Unix Epoch is already well over 32 bits.

> One could imagine taking seconds since the beginning of the era (0000-1-1), or since
> the beginning of the universe.
> For the latter, we would be in the year 13_700_002_024.

2. What other fields could be in the credential?

- Place of birth
- Place of origin
- AVS-number

Fields like `address`, `employer` will most probably be in separate credentials given out
by the commune or the employer themselves.

---
## 2 - Questions for Issuer - Setting up and creating a BBS+ signature

1. What information do you find in the credential which is sent to the holder? What keywords / algorithms do you recognize?

> Looking at the credential sent to the holder, one can think of the
> [W3C VC Data Model v1.1](https://www.w3.org/TR/vc-data-model/)
>
> ```
> // the credentialSchema is encoded using utf-8, so it can accomodate
> // all types of values.
> id: 'data:application/json;charset=utf-8,',
> // This is defined in the above document from W3C
> type: 'JsonSchemaValidator2018',
> // The proof (signature in this case) uses Bls12-381 and a BBS signature.
> type: 'Bls12381BBSSignatureDock2023',
> ```
>
> More information on BLS-12-381 can be found here, for example:
> [Bls12-381](https://hackmd.io/@benjaminion/bls12-381#Security-level)

2. How many bytes is the proof sent to the holder?

> As the proof seems to be encoded in base64, which only represents 6 bits per character,
> the number of characters needs to be multiplied by 6 and divided by 8.
> Or multiplied by 3/4 = 0.75:
> 
> 110 * 0.75 = 82.5
> 
> So probably the docknetwork/crypto library concatenates multiple base64 strings.

---
## 3 - Questions for Holder - Creating a proof

1. Explore the bbsProofVerifier to see what information is sent to the verifier

> If you print the object with
> console.dir(bbsProofForVerifier, {depth: null})
> javascript will not stop at a given depth, but print everything.
> One of the interesting parts is this:
>
>     revealedAttributes: {
>       credentialSubject: { name: 'Jack Sparrow', profession: 'IT Manager' }
>     },
>     sigType: 'Bls12381BBSSignatureDock2023'
>
> Which shows the revealed attributes sent to the verifier, as well as the signature
> type which we encountered before.

2. Is there information which could be used to de-anonymize the holder?

> As far as we can interpret the structure by visual inspection, there doesn't
> seem to be a specific value which allows the verifier to identify a specific
> holder.
> However, the revealed attributes can of course be used to de-identify a holder.

3. How long is the proof now? Why is it bigger than the proof sent by the issuer?

> The proof is now longer, 582 characters, which corresponds to 436.5 bytes.
> This is due to the fact that the BBS proof sent by the holder needs to include
> the blinding factors.
> During our hands-on workshop we saw that the size of the proof is reduced if
> more fields are revealed.
> An intuitive explanation is that BBS only blinds the fields which are NOT revealed.
> So the more fields are hidden, the longer the proof.
> And the more fields are revealed, the shorter the proof.

---
## 4 - Questions for Verifier - Verifying the proof is valid

1. Can you create a proof which cannot be validated? What did you change?

> One can think of creating another keypair for the issuer, and then using
> this new keypair to verify the BBS proof generated with the old keypair.

2. Reveal another element of the credential - what did you need to change?

> The arguments to the method `presentationBuilder.markAttributesRevealed`
> allow the holder easily to choose which fields to reveal.
> The library will include the name of the fields directly in the object
> sent to the verifier.

3. How does the verifier know the credentials to reveal?

> This can be found in the structure created by the crypto library with the
> `presentationBuilder.finalize()` call.

## 5 - Discussion - Unlinkability achieved?

1. Is there a signature or a cryptopgraphic data that is shared and could used to identify the credential holder?

> Visually inspecting the structure sent from the holder to the verifier indicates that
> there is no directly linkable value sent along.
> As written in the exercise, this is of course no mathematically sound proof,
> but from an intuition point of view it is nice to see the values changing for
> every proof.

2. Is there a part of data hashed or not that was included in the presented data that could be used to identify the credential holder?

> As in exercise 1, if the data revealed to the verifier allow to single out
> a specific holder, then they will be identified.

---

## 6 - Coding exercise - Simulate an issuer-holder-verifier setup

> Here is a list of objects to be transmitted for this exercise:
>
> - Issuer
>   - to Verifier: the public key of the newly created keypair
>   - to Holder: the result of the call to `BBSCredentialBuilder.sign().toJSON()`
>     over a secure connection
> - Holder
>   - from Issuer: can recreate the credential using `BBSCredential.fromJSON()`
>   - to Verifier: the result of the call to `PresentationBuilder.finalize().toJSON()`
> - Verifier
>   - from Holder: can recreate the proof using `Presentation.fromJSON()`
>
> It is interesting also for the issuer to create more than one credential,
> so that the verifier can try to distinguish between the different credentials.

### Questions

1. What are the conditions that revealing a field doesn't lead to linkability?

> If many holders share the same value of the field, the verifier will not be able
> to link the proofs to a specific holder.
> One can think of it like "k-anonymity", where "k" is the number of holders with the
> identical value for the field.
> Once the holder reveals more than one field, the "k" drops very rapidly to 1,
> which means that the holder can be linked.

2. How can the verifier be sure that the proof is not stale, meaning that it is not a replayed proof?

> For this to work, one can do either of:
>
> 1. Define the protocol to include a timestamp of the proof
> 2. Have the verifier send a random challenge to the holder, and then
>    verify whether the challenge has been incorporated in the proof
>
> For both possibilities, the
> [IETF-draft on BBS+](https://identity.foundation/bbs-signature/draft-irtf-cfrg-bbs-signatures.html#name-header-and-presentation-hea)
> proposes for the holder to use the attribute `presentation_header` to
> attach a proof of freshness to the proof sent to the verifier.
> 

3. Can you easily implement this? Unfortunately the docknetwork/crypto-library doesn't have a good API documentation.
   Look at the [PresentationBuilder](https://github.com/docknetwork/crypto-wasm-ts/blob/c32072b85150b6c1febadb29d2b8b7f4dbe7e40b/src/anonymous-credentials/presentation-builder.ts#L120). Which field looks like the best to use for the purpose of avoiding replays?

> Unfortunately the library doesn't follow the namings of the IETF draft standard.
> But looking at the fields, two stand out:
>
> ```
> // This can specify the reason why the proof was created, or date of the proof, or self-attested attributes (as JSON string), etc
>  _context?: string;
>  // To prevent replay attack
>  _nonce?: Uint8Array;
> ```
>
> So the `context` must correspond to the `header` in the IETF-draft, and as such
> should be set by the issuer.
> The `nonce` must then be the `presentation_header`, which can be used for a challenge
> or a timestamp.
